In [1]:
import pygdf
import numpy as np
from pygdf.dataframe import DataFrame

In [2]:
import pyarrow as pa, sys, pandas as pd

In [3]:
def readArrowToDF(source):
    reader = pa.RecordBatchStreamReader(source)
    pa_df = reader.read_all()
    return pa_df.to_pandas()

In [4]:
df = readArrowToDF("node-server/uploads/uber-dataset.arrow")

In [5]:
gpu_df = DataFrame.from_pandas(df)

In [8]:
df = DataFrame()
nelem = 10**6  # A million item
df['key1'] = np.random.randint(0, 5, nelem)
df['key2'] = np.random.randint(0, 3, nelem)
df['val1'] = np.arange(1, 1 + nelem)
df['val2'] = np.random.random(nelem)

In [4]:
df.head().to_pandas()

,key1,key2,val1,val2
0,1,0,1,0.832418
1,0,1,2,0.702720
2,2,1,3,0.814997
3,4,0,4,0.698803
4,2,2,5,0.632527


In [5]:
df.groupby(by=['key1', 'key2']).mean()

/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/

   key1 key2               val1                val2
 0    0    0 499991.13897751615   0.500500273341694
 1    0    1 498149.71224435116 0.49915224768559957
 2    0    2   500246.831658405 0.49932934396900014
 3    1    0  499929.9718701189  0.4997970229389601
 4    1    1  501476.9940390855  0.5006826483188622
 5    1    2  500412.3009169631  0.5002174350180542
 6    2    0  499810.1914909582  0.5017281713406735
 7    2    1  500080.0818064206  0.5018919081998916
 8    2    2  500726.5189073721 0.49894799193962097
 9    3    0  500682.3600508145 0.49976123494834945
10    3    1 499071.18907015416 0.49736679651401494
11    3    2  500453.4085117837   0.500835048713539
12    4    0 499907.88592142676  0.5009347385916217
13    4    1 500420.31843726116  0.5011883284453145
14    4    2  498677.7382152609 0.49821989306868103